In [2]:
%%HTML
<button onclick="$('.output_stderr').toggle();">Toggle Code</button>

In [5]:
%run common.ipynb

Folder "/tmp" will be used to save temporary dictionary and corpus.


2017-11-10 10:57:26,088 : INFO : 'pattern' package not found; tag filters are not available for English


In [10]:
# %load /home/david/code/nlp/WordSeg.py
#!/usr/bin/env python2
"""
Created on Thu Nov  9 10:00:53 2017

@author: david
"""

from jieba import posseg
import jieba
import re
import sys
import codecs
import os
from gensim import corpora, models, similarities
import multiprocessing

reload(sys)  
sys.setdefaultencoding('utf8')

'''
使用结巴进行分词，可以加载自定义词典和停用词词典，可以过滤非中文字符
segMode :
e -- 精确模式，试图将句子最精确地切开，适合文本分析，也是默认的模式
a -- 全模式，把句子中所有的可以成词的词语都扫描出来, 速度非常快，但是不能解决歧义
s -- 搜索引擎模式，在精确模式的基础上，对长词再次切分，提高召回率，适合用于搜索引擎分词
userDictPath：用户自定义词典
stopWordPath：停用词词典
isFilterNoChinese：是否过滤非中文字符，默认过滤
'''
class WordSeg(object):
    def __init__(self, segMode, userDictPath = None, stopWordPath = None, isFilterNoChinese = True):
        self._segMode = segMode
        self._userDictPath = userDictPath
        self._stopWordPath = stopWordPath
        self._isFilterNoChinese = isFilterNoChinese
        
    def filterNoChinese(self, segStr):
        pattern = re.compile(ur'[^\u4e00-\u9fa5]')
        #去掉多余的空格 
        return re.sub(r"\s{2,}", "", " ".join(pattern.split(segStr)).strip())
    
    def generateStopWords(self):
        stopWords = {}
        if not self._stopWordPath == None:
            fstop = open(self._stopWordPath, 'r')
            for word in fstop:
                #解码为unicode进行处理
                stopWords[word.strip().decode('utf-8', 'ignore')] = word.strip().decode('utf-8', 'ignore')
        return stopWords
        
    def seg(self, segStr):
        segStr = segStr.strip().decode('utf8', 'ignore')
        if not self._userDictPath == None:
            jieba.load_userdict(self._userDictPath)
            
        if self._isFilterNoChinese:
            segStr = self.filterNoChinese(segStr)
        
        #多线程分词
#        jieba.enable_parallel(5)
        wordGenList = []
        if self._segMode == 'a':
            wordGenList = [word for word in jieba.cut(segStr, cut_all=True) if word not in self.generateStopWords()]
        elif self._segMode == 's':
            wordGenList = [word for word in jieba.cut_for_search(segStr) if word not in self.generateStopWords()]
        else:
            wordGenList = [word for word in jieba.cut(segStr, cut_all=False) if word not in self.generateStopWords()]
        
        return wordGenList
    
    def posSeg(self, segStr):
        segStr = segStr.strip().decode('utf8', 'ignore')
        if not self._userDictPath == None:
            jieba.load_userdict(self._userDictPath)
            
        if self._isFilterNoChinese:
            segStr = self.filterNoChinese(segStr)
        wordPosGenList = posseg.cut(segStr, HMM=True)
        return wordPosGenList
    
    
    


In [6]:
FOLDER_DIR = '/home/david/code/kuandai/'
#设置了一大一小两个语料集，为了测试增量训练
smallRawTextPath = os.path.join(FOLDER_DIR, 'smallRaw.txt')
bigRawTextPath = os.path.join(FOLDER_DIR, 'bigRaw.txt')

smallSegTextPath = os.path.join(FOLDER_DIR, 'smallSeg.txt')
bigSegTextPath = os.path.join(FOLDER_DIR, 'bigSeg.txt')

smallBinModelPath = os.path.join(FOLDER_DIR, 'small.model.bin')
smallTxtModelPath = os.path.join(FOLDER_DIR, 'small.model.txt')

bigBinModelPath = os.path.join(FOLDER_DIR, 'big.model.bin')
bigTxtModelPath = os.path.join(FOLDER_DIR, 'big.model.txt')

In [2]:
def seg(inpath, outpath):
    wordseg = WordSeg('e', stopWordPath='/home/david/code/data/nlp/stopword.txt')
    #读进来是utf8的编码
    infile = codecs.open(inpath, 'r', encoding='utf8')
    outfile = codecs.open(outpath, 'w', encoding='utf8')
    
    #对每一行分词并输出到文件
    line = infile.readline()
    while line:
        segWords = wordseg.seg(line)
        outfile.writelines(' '.join(segWords))
        outfile.writelines('\n')
        line = infile.readline()
    
    infile.close()
    outfile.close()

In [ ]:
def train(inPath, binModelPath, TxtModelPath):
    '''
    LineSentence
    size：向量长度
    min_count：词频，低于该词频的会被过滤
    window：看前后几个词
    workers：设置并发训练时候的线程数
    '''
    #用gensim自带的方法读取文件路径来训练
#    model = models.Word2Vec(models.word2vec.LineSentence(inPath), size=20, window=2, min_count=1, workers=multiprocessing.cpu_count())
    #用yield读文件，将读取的文件内容传给gensim
    model = models.Word2Vec(inPath, size=20, window=2, min_count=1, workers=multiprocessing.cpu_count())
    
    model.save(binModelPath)
    model.wv.save_word2vec_format(TxtModelPath, binary=True)